In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from colorcloud.UFGsim2024infufg import SemanticSegmentationSimLDM, ProjectionSimVizTransform
from colorcloud.biasutti2019riu import SemanticSegmentationTask, RIUNet
from torch.nn import CrossEntropyLoss
import lightning as L
import wandb
from lightning.pytorch.loggers import WandbLogger
from datetime import datetime
from torch import nn

In [3]:
data = SemanticSegmentationSimLDM()
data.setup('fit')
epoch_steps = len(data.train_dataloader())

In [4]:
n_epochs = 5
learner = SemanticSegmentationTask(
    RIUNet(in_channels=4, hidden_channels=(64, 128, 256, 512), n_classes=13),
    CrossEntropyLoss(reduction='none'),
    data.viz_tfm, 
    total_steps=n_epochs*epoch_steps
)

In [ ]:
time = datetime.now()
timestamp = str(time.year) + '-' + str(time.month) + '-' + str(time.day) + '_' + str(time.hour) + '-' + str(time.minute) + '-' + str(time.second)
name = 'UFGSim-'+ timestamp
wandb_logger = WandbLogger(project="colorcloud", name=name, log_model="all")
wandb_logger.watch(learner.model, log="all")

In [ ]:
# train model
trainer = L.Trainer(max_epochs=n_epochs, logger=wandb_logger)
trainer.fit(learner, data)

In [ ]:
wandb.finish()